安装依赖（torch版本需要自己调整）

In [1]:
!conda install pytorch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 pytorch-cuda=12.1 -c pytorch -c nvidia -y
!pip install trl modelscope transformers addict -i https://mirrors.aliyun.com/pypi/simple/ 

Channels:
 - pytorch
 - nvidia
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



C:\Users\owen\miniconda3\Lib\site-packages\conda\base\context.py:198: FutureWarning: Adding 'defaults' to channel list implicitly is deprecated and will be removed in 25.3. 

To remove this warning, please choose a default channel explicitly with conda's regular configuration system, e.g. by adding 'defaults' to the list of channels:

  conda config --add channels defaults

For more information see https://docs.conda.io/projects/conda/en/stable/user-guide/configuration/use-condarc.html

  deprecated.topic(


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


下载模型

In [2]:
from modelscope import snapshot_download
model_dir = snapshot_download('Qwen/Qwen2.5-1.5B-Instruct')

c:\Users\owen\miniconda3\envs\llm_rl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-02-09 09:56:10,042 - modelscope - INFO - Creating symbolic link [C:\Users\owen\.cache\modelscope\hub\Qwen\Qwen2.5-1.5B-Instruct].
2025-02-09 09:56:10,042 - modelscope - WARNING - Failed to create symbolic link C:\Users\owen\.cache\modelscope\hub\Qwen\Qwen2.5-1.5B-Instruct for C:\Users\owen\.cache\modelscope\hub\Qwen\Qwen2.5-1.5B-Instruct\Qwen\Qwen2___5-1___5B-Instruct.


ref和policy模型

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 根据自己电脑~/.cache位置修改
model_name = "C:\\Users\\owen\\.cache\\modelscope\\hub\\Qwen\\Qwen2___5-1___5B-Instruct"

policy = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
ref = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

policy生成答案

In [4]:
import torch 

def gen_completion(model,query,tokenizer):
    SYSTEM='''
        Always respond in the following format:
        <reasoning>
        ...
        </reasoning>
        <answer>
        ...
        </answer>
    '''
    messages=[{'role':'system','content':SYSTEM},{'role':'user','content':query}]
    text = tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_length=5000,
            do_sample=True,
        )
    completion_ids=generated_ids[0][len(model_inputs.input_ids[0]):]
    completion_text=tokenizer.decode(completion_ids, skip_special_tokens=True)
    return model_inputs.input_ids[0],text,completion_ids,completion_text

gen_completion(policy,'小明妈妈和老婆都掉水里，小明应该先救谁?',tokenizer)

c:\Users\owen\miniconda3\envs\llm_rl\lib\site-packages\transformers\integrations\sdpa_attention.py:53: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


(tensor([151644,   8948,    271,    286,  23240,   5889,    304,    279,   2701,
           3561,    510,    286,    366,  19895,    287,    397,    286,  12236,
            286,    690,  19895,    287,    397,    286,    366,   9217,    397,
            286,  12236,    286,    690,   9217,    397,    257, 151645,    198,
         151644,    872,    198,  30709,  30858, 101935,  33108, 105367,  71268,
         100373,  52510,  69249,   3837,  30709,  30858,  99730,  60726,  99509,
         100165,     30, 151645,    198, 151644,  77091,    198],
        device='cuda:0'),
 '<|im_start|>system\n\n        Always respond in the following format:\n        <reasoning>\n        ...\n        </reasoning>\n        <answer>\n        ...\n        </answer>\n    <|im_end|>\n<|im_start|>user\n小明妈妈和老婆都掉水里，小明应该先救谁?<|im_end|>\n<|im_start|>assistant\n',
 tensor([104596, 104705,   3837,  30709,  30858,  85106, 104747, 101118, 100005,
          99464,   1773,  62244,  42411, 100006,  82894,  49828, 10144

计算model预测的目标token概率

In [5]:
def gen_probs(model,input_ids,completion_ids):
    full_ids=torch.cat([input_ids,completion_ids],dim=0).unsqueeze(0)
    output=model(full_ids)
    probs=torch.nn.functional.softmax(output.logits,dim=-1)
    prob_select=full_ids[:,1:].unsqueeze(-1)
    token_probs=torch.gather(probs[:,:-1],dim=-1,index=prob_select)[0]
    return token_probs[len(input_ids)-1:,0]

input_ids,input_text,completion_ids,completion_text=gen_completion(policy,'3+5等于几?',tokenizer)
ref_token_probs=gen_probs(ref,input_ids,completion_ids)
print(ref_token_probs.shape,completion_ids.shape,completion_text)

torch.Size([47]) torch.Size([47]) <reasoning>
加法是数学中的基本运算之一，用于计算两个数的总和。在这个问题中，我们需要将数字3和5相加。
</reasoning>

<answer>
8
</answer>


加载gsm8k数学题

In [6]:
from modelscope.msdatasets import MsDataset

# 提取答案
def extract_hash_answer(text):
    if "####" not in text:
        return ''
    return text.split("####")[1].strip()

ds =  MsDataset.load('modelscope/gsm8k', subset_name='main', split='train')
ds = ds.map(lambda x: {'final_answer':extract_hash_answer(x['answer'])})

ds[0]

2025-02-09 09:56:21,420 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from gsm8k. Please make sure that you can trust the external codes.
2025-02-09 09:56:21,687 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from modelscope/gsm8k. Please make sure that you can trust the external codes.
2025-02-09 09:56:21,689 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from modelscope/gsm8k. Please make sure that you can trust the external codes.
2025-02-09 09:56:21,690 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from modelscope/gsm8k. Please make sure that you can trust the external codes.


{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72',
 'final_answer': '72'}

奖励函数

In [7]:
def reward_func(input_text,completion_text,answer):
    score=0
    
    # 答案完全正确
    model_answer = completion_text.split("<answer>")[-1].split("</answer>")[0].strip()
    if model_answer==answer:
        score+=2
        
    # 答案只写数字
    if model_answer.isdigit():
        score+=0.5
    
    # 格式遵循
    for tag in ['<reasoning>','</reasoning>','<answer>','</answer>']:
        if completion_text.count(tag)==1:
            score+=0.125
    return score

#reward_func('','<reasoning></reasoning><answer>10</answer>','10')

GRPO训练

In [ ]:
from torch.utils.data import DataLoader
from torch.optim import SGD
import torch 

G=4 # 每个Query回答G次
dataloader=DataLoader(ds,batch_size=1,shuffle=True) 
optimizer=SGD(policy.parameters(),lr=1e-4)

while True:
    for batch in dataloader:
        batch_queries=batch['question']
        batch_answers=batch['answer']
        batch_final_answers=batch['final_answer']
                
        # 采样
        batch_ref_policy_probs=[]
        batch_completions=[]
        for q_i in range(len(batch_queries)):
            group_completions=[]
            group_ref_policy_probs=[]
            for o_i in range(G): 
                # old policy completion
                completion=gen_completion(policy,batch_queries[q_i],tokenizer)
                group_completions.append(completion)
                # ref policy probs
                input_ids,completion_ids=completion[0],completion[2]
                ref_policy_probs=gen_probs(ref,input_ids,completion_ids)
                group_ref_policy_probs.append(ref_policy_probs.detach())
            batch_completions.append(group_completions)
            batch_ref_policy_probs.append(group_ref_policy_probs)
        
        # 计算优势
        batch_group_advantages=[]
        batch_old_policy_probs=[]
        for q_i in range(len(batch_queries)): 
            group_completions=batch_completions[q_i]
            group_rewards=[]
            group_old_policy_probs=[]
            for o_i in range(G):
                # reward
                input_ids,input_text,completion_ids,completion_text=group_completions[o_i]
                reward=reward_func(input_text,completion_text,batch_final_answers[q_i])
                group_rewards.append(reward)
                # old policy probs
                old_policy_probs=gen_probs(policy,input_ids,completion_ids)
                group_old_policy_probs.append(old_policy_probs.detach())
            batch_old_policy_probs.append(group_old_policy_probs)
            
            # 组内优势计算
            group_rewards=torch.tensor(group_rewards,dtype=policy.dtype)
            group_advantages=(group_rewards-group_rewards.mean())/(group_rewards.std()+1e-8)
            batch_group_advantages.append(group_advantages)
        
        # 训练N次
        for times in range(1):
            loss=0
            for q_i in range(len(batch_queries)):
                g_loss=0
                group_completions=batch_completions[q_i]
                group_old_policy_probs=batch_old_policy_probs[q_i]
                group_advantages=batch_group_advantages[q_i]
                group_ref_policy_probs=batch_ref_policy_probs[q_i]
                for o_i in range(G):
                    input_ids,input_text,completion_ids,completion_text=group_completions[o_i]
                    # policy probs
                    policy_probs=gen_probs(policy,input_ids,completion_ids)
                    # kl
                    KL=group_ref_policy_probs[o_i]/(policy_probs+1e-8)-torch.log(group_ref_policy_probs[o_i]/(policy_probs+1e-8))-1
                    # compute loss
                    o_loss=group_advantages[o_i]*(policy_probs/(group_old_policy_probs[o_i]+1e-8))-0.1*KL # B取0.1
                    g_loss=g_loss+o_loss.sum()
                loss=loss-g_loss/G
            loss=loss/len(batch_queries)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print('loss:',loss)